In [8]:
import xml.etree.ElementTree as ET


In [11]:
stuff = ET.parse('Library.xml')
root = stuff.getroot()


In [16]:
import sqlite3
conn = sqlite3.connect('trackdb.sqlite')
cur = conn.cursor()

In [17]:
#SQL commands
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;

CREATE TABLE Artist(
id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
name TEXT UNIQUE
);

CREATE TABLE Album(
id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
artist_id INTEGER,
title TEXT UNIQUE
);

CREATE TABLE Track(
id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
title TEXT UNIQUE,
album_id INTEGER,
len INTEGER, rating INTEGER, count INTEGER
);

''')

In [18]:
def lookup(d,key):
    found = False
    for ch in d:
        if found : return ch.text
        if ch.tag == 'key' and ch.text == key:
            found = True
    return None


In [19]:
all = stuff.findall('dict/dict/dict')
print('Dict count', len(all))

Dict count 404


In [23]:
for elem in all:
    keys = elem.findall('key')
    values = elem.findall('string')
    for key, value in zip(keys, values):
        print(key.text, value.text)

Track ID Another One Bites The Dust
Name Queen
Artist John Deacon
Composer Greatest Hits
Album Rock
Genre MPEG audio file
Kind 21130E105F3B8845
Size File
Total Time file:///Users/csev/Music/iTunes/iTunes%20Music/Compilations/Greatest%20Hits/03%20Another%20One%20Bites%20The%20Dust.mp3
Track ID Asche Zu Asche
Name Rammstein
Artist Christoph Doom Schneider, Doktor Christian Lorenz, Oliver Riedel, Paul Landers, Richard Z. Kruspe-Bernstein & Till Lindemann
Composer Herzeleid
Album Industrial
Genre MPEG audio file
Kind 21130E105F3B8846
Size File
Total Time file:///Users/csev/Music/iTunes/iTunes%20Music/Rammstein/Herzeleid/04%20Asche%20Zu%20Asche.mp3
Track ID Beauty School Dropout
Name Various
Artist Grease
Album Soundtrack
Genre MPEG audio file
Kind 21130E105F3B8847
Size File
Total Time file:///Users/csev/Music/iTunes/iTunes%20Music/Various/Grease/06%20Beauty%20School%20Dropout.mp3
Track ID Black Dog
Name Led Zeppelin
Artist Jimmy Page, Robert Plant, John Paul Jones
Composer IV
Album Rock
Ge

In [26]:
for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue

    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    album = lookup(entry, 'Album')
    count = lookup(entry, 'Play Count')
    rating = lookup(entry, 'Rating')
    length = lookup(entry, 'Total Time')

    if name is None or artist is None or album is None : 
        continue

    print(f'Name is {name}, Artist is {artist}, Album is {album}, count is {count}, rating is {rating}, length is {length}\n')

    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Track
        (title, album_id, len, rating, count) 
        VALUES ( ?, ?, ?, ?, ? )''', 
        ( name, album_id, length, rating, count ) )

    conn.commit()

Name is Another One Bites The Dust, Artist is Queen, Album is Greatest Hits, count is 55, rating is 100, length is 217103

Name is Asche Zu Asche, Artist is Rammstein, Album is Herzeleid, count is 79, rating is 100, length is 231810

Name is Beauty School Dropout, Artist is Various, Album is Grease, count is 48, rating is 100, length is 239960

Name is Black Dog, Artist is Led Zeppelin, Album is IV, count is 109, rating is 100, length is 296620

Name is Bring The Boys Back Home, Artist is Pink Floyd, Album is The Wall [Disc 2], count is 33, rating is 100, length is 87118

Name is Circles, Artist is Bryan Lee, Album is Blues Is, count is 54, rating is 60, length is 355369

Name is Comfortably Numb, Artist is Pink Floyd, Album is The Wall [Disc 2], count is 36, rating is 100, length is 384130

Name is Crazy Little Thing Called Love, Artist is Queen, Album is Greatest Hits, count is 38, rating is 100, length is 163631

Name is Electric Funeral, Artist is Black Sabbath, Album is Paranoid, 

Name is You Raise Me Up, Artist is Selah, Album is Hiding Place, count is 9, rating is None, length is 302262

Name is Hold On, I'm Coming, Artist is The Canettes Blues Band, Album is On Tap & In the Can, count is 34, rating is 100, length is 282566

Name is Got my Mojo Working, Artist is The Canettes Blues Band, Album is On Tap & In the Can, count is 52, rating is 100, length is 310517

Name is Sweet Home Chicago, Artist is The Canettes Blues Band, Album is On Tap & In the Can, count is 27, rating is 100, length is 375849

Name is Heavy Love, Artist is The Canettes Blues Band, Album is On Tap & In the Can, count is 28, rating is 100, length is 253544

Name is Cold Cold Feeling, Artist is The Canettes Blues Band, Album is On Tap & In the Can, count is 43, rating is 100, length is 261929

Name is Lonely Avenue, Artist is The Canettes Blues Band, Album is On Tap & In the Can, count is 24, rating is 100, length is 292022

Name is Good Morning Little Schoolgirl, Artist is The Canettes Blue

Name is Tenderly, Artist is David Osborne, Album is Moonlight And Love Songs, count is 284, rating is None, length is 227395

Name is When I Fall In Love, Artist is David Osborne, Album is Moonlight And Love Songs, count is 281, rating is None, length is 187768

Name is Unforgettable, Artist is David Osborne, Album is Moonlight And Love Songs, count is 280, rating is None, length is 272927

Name is The Way You Look Tonight, Artist is David Osborne, Album is Moonlight And Love Songs, count is 277, rating is None, length is 247222

Name is I've Grown Accustomed To Your Face, Artist is David Osborne, Album is Moonlight And Love Songs, count is 280, rating is None, length is 148035

Name is Smoke Gets In Your Eyes, Artist is David Osborne, Album is Moonlight And Love Songs, count is 277, rating is None, length is 198504

Name is Fly Me To The Moon, Artist is David Osborne, Album is Moonlight And Love Songs, count is 275, rating is None, length is 182543

Name is Unchained Melody, Artist is

Name is Pooja Sankar: Building the Piazza Collaboration System, Artist is IEEE Computer Society, Album is Computing Conversations, count is None, rating is None, length is 496404

Name is Van Jacobson: Content-Centric Networking, Artist is IEEE Computer Society, Album is Computing Conversations, count is None, rating is None, length is 780251

Name is The Apache Software Foundation, Artist is IEEE Computer Society, Album is Computing Conversations, count is None, rating is None, length is 542484

Name is A Brief History of Packets, Artist is IEEE Computer Society, Album is Computing Conversations, count is None, rating is None, length is 1004643

Name is Discovering JavaScript Object Notation, Artist is IEEE Computer Society, Album is Computing Conversations, count is None, rating is None, length is 650971

Name is Inventing PHP, Artist is IEEE Computer Society, Album is Computing Conversations, count is None, rating is None, length is 494915

Name is Monash Museum of Computing History

Name is Waste Management, Artist is Undercover Boss, Album is Undercover Boss, Season 1, count is 1, rating is None, length is 2565232

Name is Sleep Like a Baby Tonight, Artist is U2, Album is Songs of Innocence, count is None, rating is None, length is 301641

Name is Evergreen, Artist is The Black Crowes, Album is Warpaint, count is None, rating is None, length is 261373

Name is Oh Josephine, Artist is The Black Crowes, Album is Warpaint, count is None, rating is None, length is 398907

Name is Movin' On Down the Line, Artist is The Black Crowes, Album is Warpaint, count is None, rating is None, length is 342693

Name is Wounded Bird, Artist is The Black Crowes, Album is Warpaint, count is None, rating is None, length is 263453

Name is Volcano, Artist is U2, Album is Songs of Innocence, count is None, rating is None, length is 194116

Name is Wee Who See the Deep, Artist is The Black Crowes, Album is Warpaint, count is None, rating is None, length is 290520

Name is Cedarwood Road